In [5]:
import numpy
import pandas as pd
import json
import requests
from movieDatabaseAPI import API_key #hiding private API key

# "This product uses the TMDB API but is not endorsed or certified by TMDB."
# https://developers.themoviedb.org/


In [6]:
#generate LGBTQIA keywords and id for keywords for movie search
keyword_list = ['lgbt', 'gay', 'lesbian','transgender','bisexual','intersex','queer','genderqueer',
                'non-binary','gender', 'asexual'] #tried 'aromantic' but no results

movieDataFrame = pd.DataFrame()
for lgbt_keyword in keyword_list:
    request=requests.get('https://api.themoviedb.org/3/search/keyword?api_key=' + API_key + '&query=' + lgbt_keyword)
    request_text = request.text
    data = json.loads(request_text)
    df = pd.json_normalize(data['results'])
    print(df) #look at categories and remove any bad ones
    movieDataFrame = movieDataFrame.append(df)

                    name      id
0           lgbt athlete  156331
1                   lgbt  158718
2              lgbt teen  163037
3   lgbt in the military  165614
4          lgbt activist  173669
5             black lgbt  195624
6         lgbt parenting  224000
7        indigenous lgbt  243575
8            jewish lgbt  243577
9          lgbt interest  266959
10            lgbt black  270245
11          elderly lgbt  271115
12         lgbt activism  275749
13           lgbt rights  280179
14           lgbt cinema  290394
15            lgbt child  295736
16                lgbtq+  296739
17              lgbtqia+  296795
             name      id
0     gay bashing   11137
1        gay club   40940
2      gay parent   41515
3       gay pride  173662
4     gay culture  196441
5     gay erotica  238059
6       gay youth  247821
7        gay slur  250048
8      gay cruise  257658
9       gay theme  258533
10            gay  264384
11     gay rights  264411
12    gay daddies  267435
13       

In [7]:
#remove non relevant keywords
#'gender reveal' and 'gender test' are relevant
removeFromDF = ['gay slur', 'gender', 'gender disguise', 'gender differences', 'gender swap', 'gender studies', 
     'gender inequality', 'gender violence']

movieDataFrame = movieDataFrame[~movieDataFrame['name'].isin(removeFromDF)]

In [8]:
#get all keyword ids for API call of movie title search
idList = movieDataFrame['id'].to_list()


#form full query for searching movie titles
keywordQuery = '&include_adult=true&include_video=true&with_keywords='
for idNumber in idList:
    keywordQuery = keywordQuery + str(idNumber) + '|'

#test full query to find number of pages of the query
apiFullMovieQueryTest = 'https://api.themoviedb.org/3/discover/movie?api_key='+ API_key + '&page=1' + keywordQuery
totalPages = 359

In [9]:
#grab data from each page of the query and append to final dataframe
finalMovieListDF = pd.DataFrame()
for pageNumber in range(1, totalPages+1):
    apiFullMovieQuery = 'https://api.themoviedb.org/3/discover/movie?api_key='+ API_key + '&page='+ str(pageNumber)+ keywordQuery
    requestFullMovieList = requests.get(apiFullMovieQuery)
    requestFullMovieList.json()
    requestFullMovie_text = requestFullMovieList.text
    data = json.loads(requestFullMovie_text)
    movieOnePageDF = pd.json_normalize(data['results'])
    finalMovieListDF = finalMovieListDF.append(movieOnePageDF)

finalMovieListDF.drop(['backdrop_path','poster_path'],axis=1)   

#print final dataframe to csv file, reorder columns and remove index
finalMovieListDF.to_csv('movieData.csv',columns=['id', 'title','original_title', 'original_language', 'overview', 
                                                 'release_date','popularity','vote_average', 'vote_count', 'adult', 
                                                 'video', 'genre_ids'],
                                                 index = False)

In [ ]:
#documentation definitions
'''
id - integer
title - string
original_title - string, unfortunately contains garbage characters because of non-English characters
original_language - language of the record
overview - string
release_date - string
popularity - number
vote_average - number
vote_count - integer
adult - boolean - Boolean to indicate an adult movie.
video - boolean - Boolean to indicate video.
genre_ids - array[integer]
'''